<a href="https://colab.research.google.com/github/yubraaj11/sentence_transformer_nepali/blob/master/Recommending_using_trained_sentence_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jun  5 07:10:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Defining the path for the trained and saved SentenceTransformer model to produce encodings of the scraped dataset.

In [3]:
model_path = '/content/drive/My Drive/trained_Model/sentence_transformer_nepali' 
csv_path = '/content/drive/MyDrive/Datasets/3k_News.csv'

In [4]:
import pandas as pd

In [9]:
df = pd.read_csv(csv_path, index_col=0, nrows=100)
df

,id,title,article,link
0,0,"काँक्राका फाइदै-फाइदा, कुन समयमा खानु राम्रो ?",चर्को गर्मीमा काँक्रा खानुको मज्जा नै बेग्लै ...,https://www.onlinekhabar.com/2023/05/1312396
1,1,"थाइराइडका बिरामीले के खाने, के नखाने ?",काठमाडौं । शरीरलाई राम्रोसँग काम गर्न विभिन्न...,https://www.onlinekhabar.com/2023/05/1312323
2,2,बच्चामा अन्धोपनको जोखिम बढाउने आरओपी समस्या के...,सामान्य बच्चाको तुलनामा समयअगावै जन्मिएका बच्...,https://www.onlinekhabar.com/2023/05/1312266
3,3,फुङलिङमा सवारी दुर्घटनामा परी एक बालककाे मृत्यु,"१२ जेठ, ताप्लेजुङ। स्कार्पियाे दुर्घटनामा बिह...",https://www.onlinekhabar.com/2023/05/1312637
4,4,जोर्डनका युवराज र साउदी युवतीबीचको विवाह किन छ...,"१२ जेठ, काठमाडौं । जोर्डनका युवराज हुसेन बिन ...",https://www.onlinekhabar.com/2023/05/1312564
...,...,...,...,...
95,95,भोलि काठमाडौं उपत्यकामा सार्वजनिक बिदा,"१० जेठ, काठमाडौं । भोटो देखाउने जात्राको अवसर...",https://www.onlinekhabar.com/2023/05/1311800
96,96,ग्यास र बिजुली प्रयोगको शिक्षा विद्यालयकै पाठ्...,"१० जेठ, काठमाडौं । नेपाली कांग्रेसका सांसद चन...",https://www.onlinekhabar.com/2023/05/1311778
97,97,गौतमबुद्ध विमानस्थलबाट लक्ष्यको १० प्रतिशत मात...,"१० जेठ, काठमाडौं । नेपालको दोस्रो अन्तर्राष्ट...",https://www.onlinekhabar.com/2023/05/1311777
98,98,ज्ञानेन्द्र शाहीको प्रश्न : आईजीपीलाई निलम्बन ...,"१० जेठ, काठमाडौं। राष्ट्रिय प्रजातन्त्र पार्ट...",https://www.onlinekhabar.com/2023/05/1311764


In [6]:
import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=1e2782dfecea84ab161a69e6c27c2aa8d5f446786217c3d9a8bdb35bece51686
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(model_path)
model.to(device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [10]:
embeddings = model.encode(df['article'])
embeddings.shape

(100, 768)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarities = cosine_similarity(embeddings, embeddings)

cosine_similarities.shape

(100, 100)

In [13]:
results = {}
for idx, row in df.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], df['id'][i]) for i in similar_indices]
    results[row['id']] = similar_items[1:]
print('done!')

done!


In [14]:
similar_indices[:10]

array([99,  1, 85,  0, 61, 38,  2, 96,  5, 65])

In [15]:
def item(id):
    return df.loc[df['id'] == id]['title'].tolist()[0].split(' - ')[0]

# Just reads the results out of the dictionary.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended : " + item(rec[1]) + " (score:" + str(rec[0]) + ")",end='\n\n')

recommend(item_id=10, num=10)

Recommending 10 products similar to जनकपुर ११ का वडाध्यक्ष र वडासचिवविरुद्ध भ्रष्टाचारको मुद्दा...
-------
Recommended : ढोरपाटनका मेयरलाई एमालेले गर्‍यो प्रदेश कमिटीबाट निलम्बन (score:0.8611749)

Recommended : सप्तरीमा करेन्ट लागेर एक विद्यार्थीको मृत्यु, घटनाको विरोधमा प्रदर्शन (score:0.8346316)

Recommended : सरकारी वकिलको दाबी : टोपबहादुर पनि प्रमुख योजनाकार (score:0.81892264)

Recommended : कैलालीमा लागुऔषधसहित चार जना पक्राउ (score:0.8143764)

Recommended : रेशम चौधरीलाई आममाफी नदिन राष्ट्रपति कार्यालयमा निवेदन (score:0.81278366)

Recommended : मेटमणि चौधरीले मागे अख्तियार प्रमुख राईको राजीनामा (score:0.8052186)

Recommended : खाँणलाई हिरासतबाट टेकनारायणको सन्देश : प्रधानमन्त्रीलाई भनेर अनुसन्धान रोक्नुस्, पैसा फिर्ता गरिदिनुस् (score:0.80361426)

Recommended : कांग्रेसको प्रचार विभाग प्रमुखमा विश्वकर्मा (score:0.80210096)

Recommended : नक्कली भुटानी शरणार्थी प्रकरणमा ३० जनाविरुद्ध मुद्दा (score:0.7982111)

Recommended : फुङलिङमा सवारी दुर्घटनामा परी एक बालककाे मृत्यु (score:0.7

In [16]:
def recomendation(idx,no_of_news_article):
    #get similarity values with other articles
    similarity_score = list(enumerate(cosine_similarities[idx]))
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    # Get the scores of the n most similar news articles. Ignore the first movie.
    similarity_score = similarity_score[1:no_of_news_article+1]
    
    print("Article Read -- " + df['title'].iloc[idx] +" link --"+ df['link'].iloc[idx])
    print(" ---------------------------------------------------------- ")
    news_indices = [i[0] for i in similarity_score]
    for i in range(len(news_indices)):
        print("Recomendation "+ str(i+1)+" --- " +str(news_indices[i])+"(IDX)  "+ df['title'].iloc[news_indices[i]] +" || Link --"+ df['link'].iloc[news_indices[i]] +" score -- "+ str(similarity_score[i][1]))
        print()

In [18]:
idx=11
no_of_news_article=5
recomendation(idx,no_of_news_article)

Article Read -- रेशम चौधरीलाई आममाफी नदिन राष्ट्रपति कार्यालयमा निवेदन link --https://www.onlinekhabar.com/2023/05/1312657
 ---------------------------------------------------------- 
Recomendation 1 --- 19(IDX)  रेशम चौधरीलाई आममाफी दिने प्रक्रिया रोक्न जनमोर्चाको माग || Link --https://www.onlinekhabar.com/2023/05/1312558 score -- 0.9396926

Recomendation 2 --- 30(IDX)  रेशम चौधरीको सजाय माफीको प्रस्ताव राष्ट्रपति कार्यालयमा || Link --https://www.onlinekhabar.com/2023/05/1312247 score -- 0.93701804

Recomendation 3 --- 90(IDX)  नक्कली भुटानी शरणार्थी प्रकरण : दोस्रो चरणको अनुसन्धान थाल्न प्रहरीलाई निर्देशन || Link --https://www.onlinekhabar.com/2023/05/1311882 score -- 0.93645394

Recomendation 4 --- 89(IDX)  शरणार्थी शिविरमा दलाली धन्दा चलाउन खोजिएको भन्दा गृह सचिवको जवाफ : टेन्सन नलिनू || Link --https://www.onlinekhabar.com/2023/05/1311890 score -- 0.9243067

Recomendation 5 --- 92(IDX)  ‘विदेशी नागरिकता त्यागेको निस्सा तपाईंलाई बुझाउनुपर्ने हो ?’ || Link --https://www.onlinekhabar.